#This Notebook
This notebook is based almost in its entirety on the "Delta Lake Rapid Start with Python" eLearning course.

This notebook is for students in the "Big Data Management" course, available through the materials made available though 
the Databricks University Alliance, and ***NOT*** for public distribution.

To get started, run the next cell, which will create a widget above this cell where you should enter your first name (with no spaces).

In [0]:
dbutils.widgets.text("username", "","Enter your first name")

In [0]:
import re
username =  re.sub(r"\s+", "", dbutils.widgets.get("username").strip(), flags=re.UNICODE)
if len(username) == 0:
  raise Exception("Please enter your first name in the widget at the top of the screen")
if username != dbutils.widgets.get("username"):
  raise Exception("Please enter your first name in the widget at the top of the screen without any spaces")
  
spark.sql(f"CREATE DATABASE IF NOT EXISTS dbacademy_{username}")
spark.sql(f"USE dbacademy_{username}")
health_tracker = f"/dbacademy/{username}/DLRS/healthtracker/"

In [0]:
spark.conf.set("spark.sql.shuffle.partitions", 8)

### Get the data files
The following cell used the `wget` function to retrieve files from the web.  Since the notebook is using Amazon's EC2 in the Oregon data center (kindly hosted by Databricks) and 
the files being retrieved are on S3 in the same data center, the transfer is quick (and incurs no AWS transfer charges).

In [0]:
%sh

wget https://hadoop-and-big-data.s3-us-west-2.amazonaws.com/fitness-tracker/health_tracker_data_2020_1.json
wget https://hadoop-and-big-data.s3-us-west-2.amazonaws.com/fitness-tracker/health_tracker_data_2020_2.json
wget https://hadoop-and-big-data.s3-us-west-2.amazonaws.com/fitness-tracker/health_tracker_data_2020_2_late.json
wget https://hadoop-and-big-data.s3-us-west-2.amazonaws.com/fitness-tracker/health_tracker_data_2020_3.json


--2020-11-14 02:45:13-- https://hadoop-and-big-data.s3-us-west-2.amazonaws.com/fitness-tracker/health_tracker_data_2020_1.json
Resolving hadoop-and-big-data.s3-us-west-2.amazonaws.com (hadoop-and-big-data.s3-us-west-2.amazonaws.com)... 52.218.184.9
Connecting to hadoop-and-big-data.s3-us-west-2.amazonaws.com (hadoop-and-big-data.s3-us-west-2.amazonaws.com)|52.218.184.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 310628 (303K) [application/json]
Saving to: ‘health_tracker_data_2020_1.json’

 0K .......... .......... .......... .......... .......... 16% 37.6M 0s
 50K .......... .......... .......... .......... .......... 32% 27.6M 0s
 100K .......... .......... .......... .......... .......... 49% 31.0M 0s
 150K .......... .......... .......... .......... .......... 65% 27.2M 0s
 200K .......... .......... .......... .......... .......... 82% 29.6M 0s
 250K .......... .......... .......... .......... .......... 98% 28.3M 0s
 300K ... 100% 6385G=0.01s

2020-11-14 02:45:13 (30.2 MB/s) - ‘health_tracker_data_2020_1.json’ saved [310628/310628]

--2020-11-14 02:45:13-- https://hadoop-and-big-data.s3-us-west-2.amazonaws.com/fitness-tracker/health_tracker_data_2020_2.json
Resolving hadoop-and-big-data.s3-us-west-2.amazonaws.com (hadoop-and-big-data.s3-us-west-2.amazonaws.com)... 52.218.184.9
Connecting to hadoop-and-big-data.s3-us-west-2.amazonaws.com (hadoop-and-big-data.s3-us-west-2.amazonaws.com)|52.218.184.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 284670 (278K) [application/json]
Saving to: ‘health_tracker_data_2020_2.json’

 0K .......... .......... .......... .......... .......... 17% 66.7M 0s
 50K .......... .......... .......... .......... .......... 35% 75.1M 0s
 100K .......... .......... .......... .......... .......... 53% 64.0M 0s
 150K .......... .......... .......... .......... .......... 71% 59.4M 0s
 200K .......... .......... .......... .......... .......... 89% 67.6M 0s
 250K .......... .......... ....... 100% 57.5M=0.004s

2020-11-14 02:45:13 (65.2 MB/s) - ‘health_tracker_data_2020_2.json’ saved [284670/284670]

--2020-11-14 02:45:13-- https://hadoop-and-big-data.s3-us-west-2.amazonaws.com/fitness-tracker/health_tracker_data_2020_2_late.json
Resolving hadoop-and-big-data.s3-us-west-2.amazonaws.com (hadoop-and-big-data.s3-us-west-2.amazonaws.com)... 52.218.184.9
Connecting to hadoop-and-big-data.s3-us-west-2.amazonaws.com (hadoop-and-big-data.s3-us-west-2.amazonaws.com)|52.218.184.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5827 (5.7K) [application/json]
Saving to: ‘health_tracker_data_2020_2_late.json’

 0K ..... 100% 67.2M=0s

2020-11-14 02:45:13 (67.2 MB/s) - ‘health_tracker_data_2020_2_late.json’ saved [5827/5827]

--2020-11-14 02:45:13-- https://hadoop-and-big-data.s3-us-west-2.amazonaws.com/fitness-tracker/health_tracker_data_2020_3.json
Resolving hadoop-and-big-data.s3-us-west-2.amazonaws.com (hadoop-and-big-data.s3-us-west-2.amazonaws.com)... 52.218.184.9
Connecting to hadoop-and-big-data.s3-us-west-2.amazonaws.com (hadoop-and-big-data.s3-us-west-2.amazonaws.com)|52.218.184.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 402785 (393K) [application/json]
Saving to: ‘health_tracker_data_2020_3.json’

 0K .......... .......... .......... .......... .......... 12% 49.2M 0s
 50K .......... .......... .......... .......... .......... 25% 42.8M 0s
 100K .......... .......... .......... .......... .......... 38% 50.3M 0s
 150K .......... .......... .......... .......... .......... 50% 58.3M 0s
 200K .......... .......... .......... .......... .......... 63% 51.3M 0s
 250K .......... .......... .......... .......... .......... 76% 48.6M 0s
 300K .......... .......... .......... .......... .......... 88% 31.3M 0s
 350K .......... .......... .......... .......... ... 100% 39.5M=0.009s

2020-11-14 02:45:13 (45.0 MB/s) - ‘health_tracker_data_2020_3.json’ saved [402785/402785]

In [0]:
dbutils.fs.ls("file:/databricks/driver/")

Out[8]: [FileInfo(path='file:/databricks/driver/conf/', name='conf/', size=4096),
 FileInfo(path='file:/databricks/driver/health_tracker_data_2020_2.json', name='health_tracker_data_2020_2.json', size=284670),
 FileInfo(path='file:/databricks/driver/logs/', name='logs/', size=4096),
 FileInfo(path='file:/databricks/driver/derby.log', name='derby.log', size=733),
 FileInfo(path='file:/databricks/driver/health_tracker_data_2020_1.json', name='health_tracker_data_2020_1.json', size=310628),
 FileInfo(path='file:/databricks/driver/ganglia/', name='ganglia/', size=4096),
 FileInfo(path='file:/databricks/driver/health_tracker_data_2020_3.json', name='health_tracker_data_2020_3.json', size=402785),
 FileInfo(path='file:/databricks/driver/health_tracker_data_2020_2_late.json', name='health_tracker_data_2020_2_late.json', size=5827),
 FileInfo(path='file:/databricks/driver/eventlogs/', name='eventlogs/', size=4096)]

### Move your data files into DBFS
In the following cell you are moving your data from the local file system (on your cluster's driver) to the Databricks File System (DBFS)

In [0]:
dbutils.fs.mv("file:/databricks/driver/health_tracker_data_2020_1.json", 
              health_tracker + "raw/health_tracker_data_2020_1.json")
dbutils.fs.mv("file:/databricks/driver/health_tracker_data_2020_2.json", 
              health_tracker + "raw/health_tracker_data_2020_2.json")
dbutils.fs.mv("file:/databricks/driver/health_tracker_data_2020_2_late.json", 
              health_tracker + "raw/health_tracker_data_2020_2_late.json")
dbutils.fs.mv("file:/databricks/driver/health_tracker_data_2020_3.json", 
              health_tracker + "raw/health_tracker_data_2020_3.json")

Out[9]: True

### Create a DataFrame for the first JSON file
Four JSON files were downloaded.  In the following cell you create a DataFrame for the first of those 
files and then preview 20 records using `show`.  

We will do a little bit of profiling and then move the data into a table
using the `parquet` file format.

In [0]:
file_path = health_tracker + "raw/health_tracker_data_2020_1.json"
 
health_tracker_data_2020_1_df = (
  spark.read
  .format("json")
  .load(file_path)
)
health_tracker_data_2020_1_df.show()

+---------+-------------+--------------+-----------+
device_id| heartrate| name| time|
+---------+-------------+--------------+-----------+
 0|52.8139067501|Deborah Powell|1.5778368E9|
 0|53.9078900098|Deborah Powell|1.5778404E9|
 0|52.7129593616|Deborah Powell| 1.577844E9|
 0|52.2880422685|Deborah Powell|1.5778476E9|
 0|52.5156095386|Deborah Powell|1.5778512E9|
 0|53.6280743846|Deborah Powell|1.5778548E9|
 0|52.1760037066|Deborah Powell|1.5778584E9|
 0|90.0456721836|Deborah Powell| 1.577862E9|
 0|89.4695644522|Deborah Powell|1.5778656E9|
 0|88.1490304138|Deborah Powell|1.5778692E9|
 0|86.3092976213|Deborah Powell|1.5778728E9|
 0|86.6672980008|Deborah Powell|1.5778764E9|
 0|89.2044154758|Deborah Powell| 1.57788E9|
 0|88.8691678316|Deborah Powell|1.5778836E9|
 0|87.7252170634|Deborah Powell|1.5778872E9|
 0|87.8357187561|Deborah Powell|1.5778908E9|
 0|87.1769562648|Deborah Powell|1.5778944E9|
 0|88.3403107931|Deborah Powell| 1.577898E9|
 0|88.9083484735|Deborah Powell|1.5779016E9|
 0| 87.921434203|Deborah Powell|1.5779052E9|
+---------+-------------+--------------+-----------+
only showing top 20 rows

### Visualize the data
In the following cell you can visualize the Dataframe using the `display` function.

To see a grouped bar chart, use the following plot options:
* Keys: time
* Series groupings: device_id
* Values: heartrate
* Aggregaton: SUM
* Display type: Bar chart (it's the default)

In [0]:
display(health_tracker_data_2020_1_df)

device_id heartrate name time 0 52.8139067501 Deborah Powell 1.5778368E9 0 53.9078900098 Deborah Powell 1.5778404E9 0 52.7129593616 Deborah Powell 1.577844E9 0 52.2880422685 Deborah Powell 1.5778476E9 0 52.5156095386 Deborah Powell 1.5778512E9 0 53.6280743846 Deborah Powell 1.5778548E9 0 52.1760037066 Deborah Powell 1.5778584E9 0 90.0456721836 Deborah Powell 1.577862E9 0 89.4695644522 Deborah Powell 1.5778656E9 0 88.1490304138 Deborah Powell 1.5778692E9 0 86.3092976213 Deborah Powell 1.5778728E9 0 86.6672980008 Deborah Powell 1.5778764E9 0 89.2044154758 Deborah Powell 1.57788E9 0 88.8691678316 Deborah Powell 1.5778836E9 0 87.7252170634 Deborah Powell 1.5778872E9 0 87.8357187561 Deborah Powell 1.5778908E9 0 87.1769562648 Deborah Powell 1.5778944E9 0 88.3403107931 Deborah Powell 1.577898E9 0 88.9083484735 Deborah Powell 1.5779016E9 0 87.921434203 Deborah Powell 1.5779052E9 0 88.0534361925 Deborah Powell 1.5779088E9 0 51.5421813878 Deborah Powell 1.5779124E9 0 52.1116104838 Deborah Powell 1.577916E9 0 52.1998346985 Deborah Powell 1.5779196E9 1 47.5378557652 Kristin Vasser 1.5778368E9 1 48.3496970512 Kristin Vasser 1.5778404E9 1 49.1212033115 Kristin Vasser 1.577844E9 1 47.9982802854 Kristin Vasser 1.5778476E9 1 47.841083408 Kristin Vasser 1.5778512E9 1 47.7225468025 Kristin Vasser 1.5778548E9 1 47.3041211781 Kristin Vasser 1.5778584E9 1 49.1694935562 Kristin Vasser 1.577862E9 1 80.1944436044 Kristin Vasser 1.5778656E9 1 80.0663193135 Kristin Vasser 1.5778692E9 1 80.4780281723 Kristin Vasser 1.5778728E9 1 80.1692553854 Kristin Vasser 1.5778764E9 1 80.2008095649 Kristin Vasser 1.57788E9 1 77.9520274534 Kristin Vasser 1.5778836E9 1 79.6522283544 Kristin Vasser 1.5778872E9 1 78.9239411441 Kristin Vasser 1.5778908E9 1 79.223388494 Kristin Vasser 1.5778944E9 1 79.3956852183 Kristin Vasser 1.577898E9 1 80.9319580003 Kristin Vasser 1.5779016E9 1 80.2328308016 Kristin Vasser 1.5779052E9 1 79.4119133736 Kristin Vasser 1.5779088E9 1 81.6575665079 Kristin Vasser 1.5779124E9 1 81.0531967868 Kristin Vasser 1.577916E9 1 48.5891195949 Kristin Vasser 1.5779196E9 2 64.6688142255 Sam Knopp 1.5778368E9 2 64.4950089977 Sam Knopp 1.5778404E9 2 63.8375296808 Sam Knopp 1.577844E9 2 64.2255472525 Sam Knopp 1.5778476E9 2 64.2773934482 Sam Knopp 1.5778512E9 2 107.7739656684 Sam Knopp 1.5778548E9 2 182.8913634142 Sam Knopp 1.5778584E9 2 181.8531340179 Sam Knopp 1.577862E9 2 184.7433209566 Sam Knopp 1.5778656E9 2 182.5384996807 Sam Knopp 1.5778692E9 2 106.3911944709 Sam Knopp 1.5778728E9 2 107.3676265692 Sam Knopp 1.5778764E9 2 108.8281472865 Sam Knopp 1.57788E9 2 105.2005136887 Sam Knopp 1.5778836E9 2 108.2355138599 Sam Knopp 1.5778872E9 2 106.827148526 Sam Knopp 1.5778908E9 2 106.5797252733 Sam Knopp 1.5778944E9 2 106.4873059372 Sam Knopp 1.577898E9 2 107.8284140372 Sam Knopp 1.5779016E9 2 107.0688500717 Sam Knopp 1.5779052E9 2 107.5360877314 Sam Knopp 1.5779088E9 2 105.478278919 Sam Knopp 1.5779124E9 2 106.838820333 Sam Knopp 1.577916E9 2 63.8947166524 Sam Knopp 1.5779196E9 3 55.2272036665 Minh Nguyen 1.5778368E9 3 56.035689123 Minh Nguyen 1.5778404E9 3 55.6403282219 Minh Nguyen 1.577844E9 3 56.3692513843 Minh Nguyen 1.5778476E9 3 56.5412281859 Minh Nguyen 1.5778512E9 3 55.8311481148 Minh Nguyen 1.5778548E9 3 54.9402513831 Minh Nguyen 1.5778584E9 3 92.2205431894 Minh Nguyen 1.577862E9 3 93.8159033652 Minh Nguyen 1.5778656E9 3 92.0210547557 Minh Nguyen 1.5778692E9 3 94.5958717893 Minh Nguyen 1.5778728E9 3 93.4735054948 Minh Nguyen 1.5778764E9 3 91.7998293601 Minh Nguyen 1.57788E9 3 92.9906604481 Minh Nguyen 1.5778836E9 3 95.6456366204 Minh Nguyen 1.5778872E9 3 94.9907571912 Minh Nguyen 1.5778908E9 3 94.0990083387 Minh Nguyen 1.5778944E9 3 93.0807377652 Minh Nguyen 1.577898E9 3 92.1790650046 Minh Nguyen 1.5779016E9 3 91.1192970064 Minh Nguyen 1.5779052E9 3 93.7044470734 Minh Nguyen 1.5779088E9 3 93.0486948835 Minh Nguyen 1.5779124E9 3 56.2837926147 Minh Nguyen 1.577916E9 3 55.5136669634 Minh Nguyen 1.5779196E9 4 60.7236962271 James Hou 1.5778368E9 4 59.7518

# Creating a Parquet Table
In the following steps we will load the data into a parquet table.

The first cell does some clean up  (which initially isunnecessary and will return False).  This is included in case the 
notebook is rerun

In [0]:
dbutils.fs.rm(health_tracker + "processed", recurse=True)

Out[12]: False

### Transformations
The following cell defines a function that makes the following transformations to a DataFrame:
* Use the `from_unixtime` Spark SQL function to transform the unix timestamp into a time string
* Cast the `time` column to type `timestamp` to replace the column `time`
* Cast the `time` column to type `date` to create the column `dte`
* Select the columns in the order in which we would like them to be written

The transformations are defined in a function so we can use it on each file's DataFrame (since we are loading static data, we could also have loaded multiple files into the DataFrame).

In [0]:
from pyspark.sql.functions import col, from_unixtime
 
def process_health_tracker_data(dataframe):
  return (
    dataframe
    .withColumn("time", from_unixtime("time"))
    .withColumnRenamed("device_id", "p_device_id")
    .withColumn("time", col("time").cast("timestamp"))
    .withColumn("dte", col("time").cast("date"))
    .withColumn("p_device_id", col("p_device_id").cast("integer"))
    .select("dte", "time", "heartrate", "name", "p_device_id")
    )

In [0]:
processedDF = process_health_tracker_data(health_tracker_data_2020_1_df)

# Writing the Data to the Database
* First we write the data to Parquet files - a common and popular Big Data file format
* We then load the files into the table
* We need to run MSCK to do an integrity check because the partitions did not initially get set when we loaded the data

In [0]:
(processedDF.write
 .mode("overwrite")
 .format("parquet")
 .partitionBy("p_device_id")
 .save(health_tracker + "processed"))

In [0]:
dbutils.fs.ls(health_tracker + "processed")

Out[16]: [FileInfo(path='dbfs:/dbacademy/Ramya/DLRS/healthtracker/processed/_SUCCESS', name='_SUCCESS', size=0),
 FileInfo(path='dbfs:/dbacademy/Ramya/DLRS/healthtracker/processed/p_device_id=0/', name='p_device_id=0/', size=0),
 FileInfo(path='dbfs:/dbacademy/Ramya/DLRS/healthtracker/processed/p_device_id=1/', name='p_device_id=1/', size=0),
 FileInfo(path='dbfs:/dbacademy/Ramya/DLRS/healthtracker/processed/p_device_id=2/', name='p_device_id=2/', size=0),
 FileInfo(path='dbfs:/dbacademy/Ramya/DLRS/healthtracker/processed/p_device_id=3/', name='p_device_id=3/', size=0),
 FileInfo(path='dbfs:/dbacademy/Ramya/DLRS/healthtracker/processed/p_device_id=4/', name='p_device_id=4/', size=0)]

In [0]:
%sql 

DROP TABLE IF EXISTS health_tracker_processed;

CREATE TABLE health_tracker_processed                        
USING PARQUET                
LOCATION "/dbacademy/$username/DLRS/healthtracker/processed"

In [0]:
health_tracker_processed = spark.read.table("health_tracker_processed")
health_tracker_processed.count()

Out[17]: 0

In [0]:
%sql

MSCK REPAIR TABLE health_tracker_processed

In [0]:
health_tracker_processed.count()

Out[18]: 3720

# Creating Delta Tables

In [0]:
%sql

DESCRIBE DETAIL health_tracker_processed

format id name description location createdAt lastModified partitionColumns numFiles sizeInBytes properties minReaderVersion minWriterVersion PARQUET null dbacademy_ramya.health_tracker_processed dbfs:/dbacademy/Ramya/DLRS/healthtracker/processed 2020-11-14T02:45:49.000+0000 null List(p_device_id) null null Map() null null

In [0]:
from delta.tables import DeltaTable

parquet_table = f"parquet.`{health_tracker}processed`"
partitioning_scheme = "p_device_id int"

DeltaTable.convertToDelta(spark, parquet_table, partitioning_scheme)

Out[19]: JavaObject id=o690

In [0]:
%sql

DROP TABLE IF EXISTS health_tracker_processed;

CREATE TABLE health_tracker_processed
USING DELTA
LOCATION "/dbacademy/$username/DLRS/healthtracker/processed"

In [0]:
%sql

DESCRIBE DETAIL health_tracker_processed

format id name description location createdAt lastModified partitionColumns numFiles sizeInBytes properties minReaderVersion minWriterVersion delta 0f54c08c-20e2-4101-b1c5-d02af451e48f dbacademy_ramya.health_tracker_processed null dbfs:/dbacademy/Ramya/DLRS/healthtracker/processed 2020-11-14T02:46:09.779+0000 2020-11-14T02:46:16.000+0000 List(p_device_id) 5 57098 Map() 1 2

In [0]:
health_tracker_processed = spark.read.table("health_tracker_processed")
health_tracker_processed.count()

Out[20]: 3720

In [0]:
dbutils.fs.rm(health_tracker + "gold/health_tracker_user_analytics", recurse=True)

Out[21]: False

# Creating a New Delta Table

In the following cells, we perform an aggregate query on the `health_tracker_processed` dataFrame we read from the Delta
tables in our database to generate a new DataFrame named `health_tracker_gold_user_analytics`. This new DataFrame calculates the 
average, maximum, and standard deviation of the heartrate for each device.

That new DataFrame is then written out as Delta files and then registered in the Metastore for the database.

In [0]:
from pyspark.sql.functions import col, avg, max, stddev

health_tracker_gold_user_analytics = (
  health_tracker_processed
  .groupby("p_device_id")
  .agg(avg(col("heartrate")).alias("avg_heartrate"),
       max(col("heartrate")).alias("max_heartrate"),
       stddev(col("heartrate")).alias("stddev_heartrate"))
)

In [0]:
(health_tracker_gold_user_analytics.write
 .format("delta")
 .mode("overwrite")
 .save(health_tracker + "gold/health_tracker_user_analytics"))

In [0]:
%sql

DROP TABLE IF EXISTS health_tracker_gold_user_analytics;

CREATE TABLE health_tracker_gold_user_analytics
USING DELTA
LOCATION "/dbacademy/$username/DLRS/healthtracker/gold/health_tracker_user_analytics"

### Visualize the results queried from the summarized data

This could be incorporated into a dashboard, but here we are using the `display` function to show it in the notebook.

* Keys: p_device_id
* Series groupings: None
* Values: max_heartrate, avg_heartrate, stddev_heartrate
* Aggregation: SUM
* Display Type: Bar Chart

In [0]:
display(spark.read.table("health_tracker_gold_user_analytics").orderBy("p_device_id"))

p_device_id avg_heartrate max_heartrate stddev_heartrate 0 81.21484441523789 186.4790827731 31.343789198032887 1 78.5776567337699 168.114687819 31.619679037848556 2 79.99574196662837 184.7433209566 31.408007741221997 3 82.65419819635204 171.8435388833 30.929328740004436 4 83.08377376550952 173.5770785921 34.16032267669617

# Appending Data to an Existing Delta Table

The data files we initially downloaded represented multiple months of data.  So far we have been processing the 
data for the first month and now we are going to append the data for the second month as follows:
* We load the data into a DataFrame from the JSON data file
* We then use the function we previously defined to do some transformations on the data.  Since that processing was defined in a function, we can call it again here with the DataFrame for the sencond month's data.
* The DataFame is then written out to the same Delta file, but with the `append` mode instead of the `overwrite` mode used when we initially created the file.

In [0]:
file_path = health_tracker + "raw/health_tracker_data_2020_2.json"
 
health_tracker_data_2020_2_df = (
  spark.read
  .format("json")
  .load(file_path)
)

In [0]:
processedDF = process_health_tracker_data(health_tracker_data_2020_2_df)

In [0]:
(processedDF.write
 .mode("append")
 .format("delta")
 .save(health_tracker + "processed"))

# Time Travel

We just appended the data for the second month, but the initial data for the first month is version 0, so 
we can use the time travel feature in Delta Lake to load just that version of the data into a dataFrame and work with it.

That first month's data had hourly measurements for 5 devices, for 31 days, so a total of 3,720 records (5 devices x 24 hours x 31 days)

In [0]:
(spark.read
 .option("versionAsOf", 0)
 .format("delta")
 .load(health_tracker + "processed")
 .count())

Out[28]: 3720

### Retrieving the latest version of the data

If we read the data into a DataFrame without specifying a version, we get the latest version of the data, which includes two months of data after we appended the second month's data.

Since the second month's data is for February (but in a leap year), we should have 29 additional days of data, or 7,200 records since we should have data for 60 days. However, we are missing 72 records.

In [0]:
(spark.read
 .format("delta")
 .load(health_tracker + "processed")
 .count())

Out[29]: 7128

### Plotting the device totals
We can query for the number of records for each device; which should be 1,440.
This shows us that device 4 is missing data.  For the 5 devices, we can view the data as a table or click on the bar chart icon to see a simple bar chart (hover over a bar to see the exact value).

If we then compare the data for devices 3 and 4, we can plot each day in a bar chart and see we are missing data for the last couple days of the month for device 4. For the second visulaization, use the following options:

* Keys: dte
* Series groupings: p_device_id
* Values: heartrate
* Aggregation: COUNT
* Display Type: Bar Chart (grouped)

Confirm to run the query over the entire dataset.

In [0]:
from pyspark.sql.functions import count

display(
  spark.read
  .format("delta")
  .load(health_tracker + "processed")
  .groupby("p_device_id")
  .count()
  .orderBy("p_device_id"))

p_device_id count 0 1440 1 1440 2 1440 3 1440 4 1368

In [0]:
display(
  spark.read
  .format("delta")
  .load(health_tracker + "processed")
  .where(col("p_device_id").isin([3,4]))
)

dte time heartrate name p_device_id 2020-01-01 2020-01-01T00:00:00.000+0000 55.2272036665 Minh Nguyen 3 2020-01-01 2020-01-01T01:00:00.000+0000 56.035689123 Minh Nguyen 3 2020-01-01 2020-01-01T02:00:00.000+0000 55.6403282219 Minh Nguyen 3 2020-01-01 2020-01-01T03:00:00.000+0000 56.3692513843 Minh Nguyen 3 2020-01-01 2020-01-01T04:00:00.000+0000 56.5412281859 Minh Nguyen 3 2020-01-01 2020-01-01T05:00:00.000+0000 55.8311481148 Minh Nguyen 3 2020-01-01 2020-01-01T06:00:00.000+0000 54.9402513831 Minh Nguyen 3 2020-01-01 2020-01-01T07:00:00.000+0000 92.2205431894 Minh Nguyen 3 2020-01-01 2020-01-01T08:00:00.000+0000 93.8159033652 Minh Nguyen 3 2020-01-01 2020-01-01T09:00:00.000+0000 92.0210547557 Minh Nguyen 3 2020-01-01 2020-01-01T10:00:00.000+0000 94.5958717893 Minh Nguyen 3 2020-01-01 2020-01-01T11:00:00.000+0000 93.4735054948 Minh Nguyen 3 2020-01-01 2020-01-01T12:00:00.000+0000 91.7998293601 Minh Nguyen 3 2020-01-01 2020-01-01T13:00:00.000+0000 92.9906604481 Minh Nguyen 3 2020-01-01 2020-01-01T14:00:00.000+0000 95.6456366204 Minh Nguyen 3 2020-01-01 2020-01-01T15:00:00.000+0000 94.9907571912 Minh Nguyen 3 2020-01-01 2020-01-01T16:00:00.000+0000 94.0990083387 Minh Nguyen 3 2020-01-01 2020-01-01T17:00:00.000+0000 93.0807377652 Minh Nguyen 3 2020-01-01 2020-01-01T18:00:00.000+0000 92.1790650046 Minh Nguyen 3 2020-01-01 2020-01-01T19:00:00.000+0000 91.1192970064 Minh Nguyen 3 2020-01-01 2020-01-01T20:00:00.000+0000 93.7044470734 Minh Nguyen 3 2020-01-01 2020-01-01T21:00:00.000+0000 93.0486948835 Minh Nguyen 3 2020-01-01 2020-01-01T22:00:00.000+0000 56.2837926147 Minh Nguyen 3 2020-01-01 2020-01-01T23:00:00.000+0000 55.5136669634 Minh Nguyen 3 2020-01-02 2020-01-02T00:00:00.000+0000 59.1128960886 Minh Nguyen 3 2020-01-02 2020-01-02T01:00:00.000+0000 57.9965743832 Minh Nguyen 3 2020-01-02 2020-01-02T02:00:00.000+0000 57.048983018 Minh Nguyen 3 2020-01-02 2020-01-02T03:00:00.000+0000 58.6076680636 Minh Nguyen 3 2020-01-02 2020-01-02T04:00:00.000+0000 58.1507310905 Minh Nguyen 3 2020-01-02 2020-01-02T05:00:00.000+0000 58.3062011069 Minh Nguyen 3 2020-01-02 2020-01-02T06:00:00.000+0000 57.5506524921 Minh Nguyen 3 2020-01-02 2020-01-02T07:00:00.000+0000 58.5553088461 Minh Nguyen 3 2020-01-02 2020-01-02T08:00:00.000+0000 95.403553178 Minh Nguyen 3 2020-01-02 2020-01-02T09:00:00.000+0000 97.0354779896 Minh Nguyen 3 2020-01-02 2020-01-02T10:00:00.000+0000 96.483197159 Minh Nguyen 3 2020-01-02 2020-01-02T11:00:00.000+0000 95.9571925121 Minh Nguyen 3 2020-01-02 2020-01-02T12:00:00.000+0000 97.4998011617 Minh Nguyen 3 2020-01-02 2020-01-02T13:00:00.000+0000 96.6868620293 Minh Nguyen 3 2020-01-02 2020-01-02T14:00:00.000+0000 96.5168010166 Minh Nguyen 3 2020-01-02 2020-01-02T15:00:00.000+0000 98.5795034683 Minh Nguyen 3 2020-01-02 2020-01-02T16:00:00.000+0000 96.5318068703 Minh Nguyen 3 2020-01-02 2020-01-02T17:00:00.000+0000 96.0814634646 Minh Nguyen 3 2020-01-02 2020-01-02T18:00:00.000+0000 96.5623086182 Minh Nguyen 3 2020-01-02 2020-01-02T19:00:00.000+0000 97.5394488685 Minh Nguyen 3 2020-01-02 2020-01-02T20:00:00.000+0000 98.0060596866 Minh Nguyen 3 2020-01-02 2020-01-02T21:00:00.000+0000 97.5373518929 Minh Nguyen 3 2020-01-02 2020-01-02T22:00:00.000+0000 95.9352482104 Minh Nguyen 3 2020-01-02 2020-01-02T23:00:00.000+0000 56.9012962691 Minh Nguyen 3 2020-01-03 2020-01-03T00:00:00.000+0000 53.0135530338 Minh Nguyen 3 2020-01-03 2020-01-03T01:00:00.000+0000 51.9593429471 Minh Nguyen 3 2020-01-03 2020-01-03T02:00:00.000+0000 53.4554769196 Minh Nguyen 3 2020-01-03 2020-01-03T03:00:00.000+0000 52.8861381062 Minh Nguyen 3 2020-01-03 2020-01-03T04:00:00.000+0000 51.9996123291 Minh Nguyen 3 2020-01-03 2020-01-03T05:00:00.000+0000 51.0225403355 Minh Nguyen 3 2020-01-03 2020-01-03T06:00:00.000+0000 88.1478480363 Minh Nguyen 3 2020-01-03 2020-01-03T07:00:00.000+0000 87.8105301315 Minh Nguyen 3 2020-01-03 2020-01-03T08:00:00.000+0000 89.4919596116 Minh Nguyen 3 2020-01-03 2020-01-03T09:00:00.000+0000 88.5433678159 Minh Nguyen 3 2020-01-03 2020-01-03T10

# Quantifying the Bad Data
In the initial data, there were some readings where the device appears to be broken since there are negative heartrate readings. Here we will refer to these as broken records.

In the following cells we will create a DataFrame containing a count by day of the records where the heartrate is less than 0 and then display those results in a tabular format.  We then get a total of the number of bad readings in the data. 

We will also create a temporary view for the `broken_readings` DataFrame.  You may be wondering why, but since checking for such 
bad data would likely be a repeated query (and we will do it again later), ***we do not have to define the query again since the temporary view is a pointer to the query and not the actual data***.

In [0]:
broken_readings = (
  health_tracker_processed
  .select(col("heartrate"), col("dte"))
  .where(col("heartrate") < 0)
  .groupby("dte")
  .agg(count("heartrate"))
  .withColumnRenamed("count(heartrate)","invalid_readings")
  .orderBy("dte")
)

broken_readings.createOrReplaceTempView("broken_readings")

In [0]:
display(broken_readings)

dte invalid_readings 2020-01-01 1 2020-01-02 1 2020-01-04 1 2020-01-06 1 2020-01-07 1 2020-01-09 2 2020-01-12 3 2020-01-13 2 2020-01-14 1 2020-01-16 2 2020-01-19 2 2020-01-20 1 2020-01-21 2 2020-01-22 2 2020-01-23 2 2020-01-24 1 2020-01-25 1 2020-01-26 1 2020-01-27 3 2020-01-28 1 2020-01-29 1 2020-01-30 1 2020-01-31 1 2020-02-01 1 2020-02-05 2 2020-02-07 2 2020-02-09 1 2020-02-12 2 2020-02-13 1 2020-02-14 2 2020-02-16 2 2020-02-17 1 2020-02-18 1 2020-02-19 1 2020-02-23 1 2020-02-24 2 2020-02-25 2 2020-02-26 2 2020-02-27 1 2020-02-29 2

In [0]:
broken_readings.select("invalid_readings").groupby().sum().show()

+---------------------+
sum(invalid_readings)|
+---------------------+
 60|
+---------------------+

# Repairing Records with "Upserts"

We saw in prior steps that you can append data to your Delta files and Delta Lake can track the versions of the data using the Metastore.

However, in the prior step we saw that we also had some bad records and missing data.  These can be addressed with upserts, which as it sounds, is a combination of the words "update" and "insert"; this is what it does. An upsert will update records where some criteria are met and otherwise will insert the record.   

In the prior steps we discovered the following data issues:
* 72 records were missing for device 4 in the data for the second month
* 60 measurements had bad data in that there were negative heartrate measurements

w will address the bad readings first by interpolating the values based on the readings prior to and subsequent to the bad reading.  This will be done using the `LAG` and `LEAD` functions in PySpark which allow us to look at the prior and subsequent measurements.  In the next cell we create the `interpolatedDF` DataFrame which includes all of the data in the table and gets the data for each measurement as well as the data for the prior and subsequent record's heartrate measurements.

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lag, lead
 
dteWindow = Window.partitionBy("p_device_id").orderBy("dte")
 
interpolatedDF = (
  spark.read
  .table("health_tracker_processed")
  .select("dte",
          "time",
          "heartrate",
          lag("heartrate").over(dteWindow).alias("prev_amt"),
          lead("heartrate").over(dteWindow).alias("next_amt"),
          "name",
          "p_device_id")
)
interpolatedDF.filter("heartrate < 0").show(truncate=False)

+----------+-------------------+---------------+--------------+--------------+--------------+-----------+
dte |time |heartrate |prev_amt |next_amt |name |p_device_id|
+----------+-------------------+---------------+--------------+--------------+--------------+-----------+
2020-01-16|2020-01-16 04:00:00|-186.8528776773|52.3973750622 |53.8522267004 |Deborah Powell|0 |
2020-01-21|2020-01-21 13:00:00|-198.6755334103|104.8835106245|105.6693337464|Deborah Powell|0 |
2020-01-21|2020-01-21 20:00:00|-7.7406103121 |106.4850184671|62.520345867 |Deborah Powell|0 |
2020-01-22|2020-01-22 14:00:00|-5.7846500868 |82.9261468212 |84.3038812018 |Deborah Powell|0 |
2020-01-24|2020-01-24 01:00:00|-12.1350515935 |65.9234725873 |64.5736710603 |Deborah Powell|0 |
2020-01-30|2020-01-30 07:00:00|-158.2001553885|94.4195145531 |93.123403291 |Deborah Powell|0 |
2020-02-05|2020-02-05 13:00:00|-102.5473513325|80.2730636814 |80.5116990516 |Deborah Powell|0 |
2020-02-17|2020-02-17 20:00:00|-155.3088950296|96.0591009206 |58.4248698302 |Deborah Powell|0 |
2020-02-23|2020-02-23 15:00:00|-123.0675085559|110.3267835162|109.930286771 |Deborah Powell|0 |
2020-02-26|2020-02-26 01:00:00|-55.6008851769 |101.5087712128|100.2589349755|Deborah Powell|0 |
2020-02-26|2020-02-26 11:00:00|-29.1636882978 |99.3939373846 |100.817988109 |Deborah Powell|0 |
2020-02-29|2020-02-29 23:00:00|-218.062685331 |54.678015242 |null |Deborah Powell|0 |
2020-01-02|2020-01-02 11:00:00|-183.9746809207|79.8507376201 |79.0430573027 |Kristin Vasser|1 |
2020-01-04|2020-01-04 20:00:00|-211.175152794 |92.6585444964 |91.5561127087 |Kristin Vasser|1 |
2020-01-06|2020-01-06 23:00:00|-113.1686910275|53.0466770469 |57.8250317365 |Kristin Vasser|1 |
2020-01-07|2020-01-07 22:00:00|-60.0372855343 |98.773501162 |58.3327903805 |Kristin Vasser|1 |
2020-01-13|2020-01-13 04:00:00|-165.0684031332|53.3585582298 |53.1340931062 |Kristin Vasser|1 |
2020-01-13|2020-01-13 18:00:00|-71.8674491009 |89.1571710243 |86.5655772085 |Kristin Vasser|1 |
2020-01-22|2020-01-22 05:00:00|-20.6071411977 |55.1367163154 |54.9189061107 |Kristin Vasser|1 |
2020-01-27|2020-01-27 18:00:00|-4.1300063426 |94.458619833 |97.6607829035 |Kristin Vasser|1 |
+----------+-------------------+---------------+--------------+--------------+--------------+-----------+
only showing top 20 rows

### Create the update DataFrame
In the following cell we create a DataFrame that will be used to update the records shown above.

Admittedly, this is a somewhat simplified approach in that there is the possibility of consecutive missing measurements, or you may have noticed in the 20 records shown above that there is a bad measurement at the end of the month (so the subsequent record has not yet been loaded and the `LEAD` value is null).

In [0]:
updatesDF = (
  interpolatedDF
  .filter("heartrate < 0")
  .select("dte",
          "time",
          ((col("prev_amt") + col("next_amt"))/2).alias("heartrate"),
          "name",
          "p_device_id")
)
print("update count:", updatesDF.count() )
updatesDF.show(60,truncate=False)

update count: 60
+----------+-------------------+------------------+--------------+-----------+
dte |time |heartrate |name |p_device_id|
+----------+-------------------+------------------+--------------+-----------+
2020-01-16|2020-01-16 04:00:00|53.124800881300004|Deborah Powell|0 |
2020-01-21|2020-01-21 13:00:00|105.27642218545 |Deborah Powell|0 |
2020-01-21|2020-01-21 20:00:00|84.50268216705001 |Deborah Powell|0 |
2020-01-22|2020-01-22 14:00:00|83.6150140115 |Deborah Powell|0 |
2020-01-24|2020-01-24 01:00:00|65.2485718238 |Deborah Powell|0 |
2020-01-30|2020-01-30 07:00:00|93.77145892205 |Deborah Powell|0 |
2020-02-05|2020-02-05 13:00:00|80.3923813665 |Deborah Powell|0 |
2020-02-17|2020-02-17 20:00:00|77.2419853754 |Deborah Powell|0 |
2020-02-23|2020-02-23 15:00:00|110.1285351436 |Deborah Powell|0 |
2020-02-26|2020-02-26 01:00:00|100.88385309415 |Deborah Powell|0 |
2020-02-26|2020-02-26 11:00:00|100.1059627468 |Deborah Powell|0 |
2020-02-29|2020-02-29 23:00:00|null |Deborah Powell|0 |
2020-01-02|2020-01-02 11:00:00|79.4468974614 |Kristin Vasser|1 |
2020-01-04|2020-01-04 20:00:00|92.10732860255 |Kristin Vasser|1 |
2020-01-06|2020-01-06 23:00:00|55.435854391700005|Kristin Vasser|1 |
2020-01-07|2020-01-07 22:00:00|78.55314577125 |Kristin Vasser|1 |
2020-01-13|2020-01-13 04:00:00|53.246325668 |Kristin Vasser|1 |
2020-01-13|2020-01-13 18:00:00|87.8613741164 |Kristin Vasser|1 |
2020-01-22|2020-01-22 05:00:00|55.02781121305 |Kristin Vasser|1 |
2020-01-27|2020-01-27 18:00:00|96.05970136825 |Kristin Vasser|1 |
2020-02-01|2020-02-01 10:00:00|88.26517343345 |Kristin Vasser|1 |
2020-02-13|2020-02-13 14:00:00|78.5661485906 |Kristin Vasser|1 |
2020-02-19|2020-02-19 12:00:00|93.12782164055 |Kristin Vasser|1 |
2020-02-24|2020-02-24 07:00:00|93.38899214989999 |Kristin Vasser|1 |
2020-02-25|2020-02-25 00:00:00|57.1503445397 |Kristin Vasser|1 |
2020-02-27|2020-02-27 04:00:00|56.8548186619 |Kristin Vasser|1 |
2020-01-09|2020-01-09 15:00:00|86.8898895492 |Sam Knopp |2 |
2020-01-12|2020-01-12 19:00:00|97.2776487597 |Sam Knopp |2 |
2020-01-16|2020-01-16 08:00:00|89.99588521375 |Sam Knopp |2 |
2020-01-19|2020-01-19 14:00:00|82.91514363895 |Sam Knopp |2 |
2020-01-19|2020-01-19 16:00:00|82.3162964004 |Sam Knopp |2 |
2020-01-23|2020-01-23 05:00:00|79.95350213255 |Sam Knopp |2 |
2020-01-25|2020-01-25 18:00:00|85.967908099 |Sam Knopp |2 |
2020-01-28|2020-01-28 17:00:00|80.1150337579 |Sam Knopp |2 |
2020-02-07|2020-02-07 06:00:00|72.6475692046 |Sam Knopp |2 |
2020-02-07|2020-02-07 12:00:00|90.4510294842 |Sam Knopp |2 |
2020-02-12|2020-02-12 11:00:00|88.95762867285 |Sam Knopp |2 |
2020-02-16|2020-02-16 13:00:00|102.43517663029999|Sam Knopp |2 |
2020-02-18|2020-02-18 08:00:00|87.58196027245 |Sam Knopp |2 |
2020-02-24|2020-02-24 12:00:00|89.02247149305 |Sam Knopp |2 |
2020-01-20|2020-01-20 08:00:00|93.4727750107 |Minh Nguyen |3 |
2020-01-26|2020-01-26 21:00:00|86.63878529435 |Minh Nguyen |3 |
2020-01-27|2020-01-27 15:00:00|75.584667015 |Minh Nguyen |3 |
2020-01-29|2020-01-29 14:00:00|101.44713704835 |Minh Nguyen |3 |
2020-02-14|2020-02-14 02:00:00|50.021687103000005|Minh Nguyen |3 |
2020-02-25|2020-02-25 07:00:00|75.72901853075 |Minh Nguyen |3 |
2020-02-29|2020-02-29 08:00:00|96.82099701855 |Minh Nguyen |3 |
2020-01-01|2020-01-01 19:00:00|101.21254699240001|James Hou |4 |
2020-01-09|2020-01-09 04:00:00|104.77145415449999|James Hou |4 |
2020-01-12|2020-01-12 02:00:00|63.68804357805 |James Hou |4 |
2020-01-12|2020-01-12 08:00:00|105.83667578194999|James Hou |4 |
2020-01-14|2020-01-14 06:00:00|66.19456476845 |James Hou |4 |
2020-01-23|2020-01-23 14:00:00|88.4576834482 |James Hou |4 |
2020-01-27|2020-01-27 22:00:00|85.21601181835 |James Hou |4 |
2020-01-31|2020-01-31 07:00:00|79.8279460276 |James Hou |4 |
2020-02-05|2020-02-05 19:00:00|170.67616483724998|James Hou |4 |
2020-02-09|2020-02-09 10:00:00|125.04163535115 |James Hou |4 |
2020-02-12|2020-02-12 15:00:00|94.65349099630001 |James Hou |4 |
2020-02-14|2020-02-14 19:00:00|108.9631148499 |James Hou |4 |
2020-

### Create the insert DataFrame
For purposes of this exercise, you may recall that there was a file named `health_tracker_data_2020_2_late.json`, and we will treat that as late-arriving data for purposes of creating our upsert.

* As with the original data, we first read the file into a DataFrame
* Using the function defined earlier, we perform the same data transformations prior to loading the data.
* We get a record count to compare to the count of missing records.

In [0]:
file_path = health_tracker + "raw/health_tracker_data_2020_2_late.json"
 
health_tracker_data_2020_2_late_df = (
  spark.read
  .format("json")
  .load(file_path)
)

In [0]:
insertsDF = process_health_tracker_data(health_tracker_data_2020_2_late_df)
print("insert record count:", insertsDF.count() )

insert record count: 72

### Combining the updates and inserts
We are going to combine our update and insert DataFrames using a `UNION`, but before we do, we will print the schemas for both the update and insert DataFrames.

In [0]:
updatesDF.printSchema()
insertsDF.printSchema()

root
-- dte: date (nullable = true)
-- time: timestamp (nullable = true)
-- heartrate: double (nullable = true)
-- name: string (nullable = true)
-- p_device_id: integer (nullable = true)

root
-- dte: date (nullable = true)
-- time: timestamp (nullable = true)
-- heartrate: double (nullable = true)
-- name: string (nullable = true)
-- p_device_id: integer (nullable = true)

In [0]:
upsertsDF = updatesDF.union(insertsDF)
print ("Upsert record count:", upsertsDF.count() )

Upsert record count: 132

### Performing the upsert
The actual update and insert is done using the `MERGE` operation, and as part of this, we specify the following:
* The condition for matching records that are an update
* What fields get updated when ther's a match
* A dictionary with name:value pairs of the fields in the database mapped to the upsert file

In [0]:
processedDeltaTable = DeltaTable.forPath(spark, health_tracker + "processed")

update_match = """
  health_tracker.time = upserts.time 
  AND 
  health_tracker.p_device_id = upserts.p_device_id
"""

update = { "heartrate" : "upserts.heartrate" }

insert = {
  "p_device_id" : "upserts.p_device_id",
  "heartrate" : "upserts.heartrate",
  "name" : "upserts.name",
  "time" : "upserts.time",
  "dte" : "upserts.dte"
}

(processedDeltaTable.alias("health_tracker")
 .merge(upsertsDF.alias("upserts"), update_match)
 .whenMatchedUpdate(set=update)
 .whenNotMatchedInsert(values=insert)
 .execute())

### Reviewing the versions using Time Travel
As we did earlier, we can still look back at only the first month using `versionAsOf` set to 0, we can also see the version after the second month is loaded (version 1 which is missing 72 records), or the current version (2) after the upsert.

In [0]:
print ("version 1 (after month 2):", 
  (spark.read
 .option("versionAsOf", 1)
 .format("delta")
 .load(health_tracker + "processed")
 .count()) )

version 1 (after month 2): 7128

In [0]:
print ("version 2 (after the upsert):", 
  (spark.read
 .option("versionAsOf", 2)
 .format("delta")
 .load(health_tracker + "processed")
 .count()) )

version 2 (after the upsert): 7200

### Using `history()` to track the provenance of the table
The `history` command allows us to see provenance information about each version of the table.  The version numbers in the results are the version numbers we used above with the `versionAsOf`option when retrieving the data.

In [0]:
display(processedDeltaTable.history())

version timestamp userId userName operation operationParameters job notebook clusterId readVersion isolationLevel isBlindAppend operationMetrics userMetadata 2 2020-11-14T02:47:58.000+0000 6469525191966356 ramyasree.mallubhatla@sjsu.edu MERGE Map(predicate -> ((health_tracker.`time` = upserts.`time`) AND (health_tracker.`p_device_id` = upserts.`p_device_id`)), matchedPredicates -> [{"actionType":"update"}], notMatchedPredicates -> [{"actionType":"insert"}]) null List(1569613669975775) 1114-024003-chain719 1 WriteSerializable false Map(numTargetRowsCopied -> 7068, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 5, numTargetRowsInserted -> 72, numTargetRowsUpdated -> 60, numOutputRows -> 7200, numSourceRows -> 132, numTargetFilesRemoved -> 10) null 1 2020-11-14T02:47:07.000+0000 6469525191966356 ramyasree.mallubhatla@sjsu.edu WRITE Map(mode -> Append, partitionBy -> []) null List(1569613669975775) 1114-024003-chain719 0 WriteSerializable true Map(numFiles -> 5, numOutputBytes -> 52877, numOutputRows -> 3408) null 0 2020-11-14T02:46:16.000+0000 6469525191966356 ramyasree.mallubhatla@sjsu.edu CONVERT Map(numFiles -> 5, partitionedBy -> ["p_device_id"], collectStats -> true) null List(1569613669975775) 1114-024003-chain719 -1 Serializable false Map(numConvertedFiles -> 5) null

# Performing a Second Upsert
After adding our new data, we could have loaded more "broken" records with bad data, so we can run our query for broken data again on the updated table.  Since we created a temporary view previously, we can use SQL to run that query again using our temporary view.

In [0]:
%sql 

SELECT SUM(`invalid_readings`) FROM broken_readings

sum(invalid_readings) 1

### Verifying these are new bad data records and then generating the new update file

**NOTE:** Since DataFrames are evaluated when needed, we can show the `broken_readings` DataFrame again and it will show us the records that are invalid (since it's reading from the same table we read from before).

For this same reason, we can then use the `updatesDF` DataFrame to get the updates for the new bad data.

We then perform the upsert as we did before and again count the number of bad records in the updated table.

In [0]:
broken_readings.show(truncate=False)

+----------+----------------+
dte |invalid_readings|
+----------+----------------+
2020-02-27|1 |
+----------+----------------+

In [0]:
print("Number of updates:", updatesDF.count() )

Number of updates: 1

In [0]:
upsertsDF = updatesDF

(processedDeltaTable.alias("health_tracker")
 .merge(upsertsDF.alias("upserts"), update_match)
 .whenMatchedUpdate(set=update)
 .whenNotMatchedInsert(values=insert)
 .execute())

In [0]:
%sql 

SELECT SUM(`invalid_readings`) FROM broken_readings

sum(invalid_readings) null

# Schema Evolution
In an Enterprise Decision Support System (EDSS), schema evolution can occur due to updates in the data source. 

For the simulated heartrate monitor data we have been using, we will assume the data now has a new field named `device_type`
that indicates the type of monitor being used.

As before, we will load the latest data from a JSON file into a DataFrame.

In [0]:
file_path = health_tracker + "raw/health_tracker_data_2020_3.json"

health_tracker_data_2020_3_df = (
  spark.read
  .format("json")
  .load(file_path)
)

### Transforming the data
As we did with the prior files, we need to transform the data for some of the date fields and to make the `p_device_id` field an integer.
However, since we now have a new field, we need to redefine our function for performing the transformation.

After transforming our data, we want to append it to our existing data as we have done before when adding a new month's data.

In [0]:
def process_health_tracker_data(dataframe):
  return (
    dataframe
    .withColumn("time", from_unixtime("time"))
    .withColumnRenamed("device_id", "p_device_id")
    .withColumn("time", col("time").cast("timestamp"))
    .withColumn("dte", col("time").cast("date"))
    .withColumn("p_device_id", col("p_device_id").cast("integer"))
    .select("dte", "time", "device_type", "heartrate", "name", "p_device_id")
    )
  
processedDF = process_health_tracker_data(health_tracker_data_2020_3_df)

In [0]:
(processedDF.write
 .mode("append")
 .format("delta")
 .save(health_tracker + "processed"))

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1569613669975857> in <module> 
 2 . mode ( "append" ) 
 3 . format ( "delta" ) 
 ----> 4 .save(health_tracker + "processed"))
 
 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 825 self . _jwrite . save ( ) 
 826 else : 
 --> 827 self . _jwrite . save ( path ) 
 828 
 829 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 AnalysisException : A schema mismatch detected when writing to the Delta table (Table ID: 0f54c08c-20e2-4101-b1c5-d02af451e48f).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- dte: date (nullable = true)
-- time: timestamp (nullable = true)
-- heartrate: double (nullable = true)
-- name: string (nullable = true)
-- p_device_id: integer (nullable = true)


Data schema:
root
-- dte: date (nullable = true)
-- time: timestamp (nullable = true)
-- device_type: string (nullable = true)
-- heartrate: double (nullable = true)
-- name: string (nullable = true)
-- p_device_id: integer (nullable = true)

 ;

### Schema validation issues
If you are familiar with relational databases, you will be aware that the data being inserted must match the existing schema of the database.
A safeguard in Delta Datalake is that if the schema does not match when we try to doo an append, the above exception will be generated.

However, we can accomodate schema evolution using the `mergeSchema` option to merge the schema of the data being appended with the schema for the existing data.

After performing the append using the `mergeSchema` option, we can do a count of the table again to make sure we have the expected number of records:<br/> (31 + 29 + 31 days) x 24 hours/day x 5 devices = 10,920

In [0]:
(processedDF.write
 .mode("append")
 .option("mergeSchema", True)
 .format("delta")
 .save(health_tracker + "processed"))

In [0]:
print("record count:", health_tracker_processed.count() )

record count: 10920

### Reviewing the updated schema
We can see that the `device_type` field has been added to the schema.

In [0]:
%sql

DESCRIBE TABLE health_tracker_processed;

col_name data_type comment dte date time timestamp heartrate double name string p_device_id int device_type string # Partitioning Part 0 p_device_id

# Deleting Data and Recovering Lost Data

Under the GDPR in European Union and CCPA in California, a user could request that all of their data be deleted.

As an example, we will delete all of the data associated with device #4.

In [0]:
processedDeltaTable.delete("p_device_id = 4")

### Data recovery

Assume that the user of device #4 did not want all of their data deleted, but wanted their name to scrubbed from the data.  In that 
case we can use Time Travel to recover their records, but set their name to null (we will leave it to the lawyers to decide for now if our prior delete really matched the intent of the GDPR or CCPA):
* First, we can retrieve version 4 of the data (the last update before the delete) for device 4, setting the user's name to None
* We then need to create the dictionary mapping of the table names to the update DataFrame (we cannot use the prior mapping because we have since evolved the schema to include the device type)
* We then process the update by merging the `upsertsDF` Dataframe into the table
* We should confirm that the record count should still be the same as after version 4 (when we loaded the 3rd month)
* We can query for the records for device 4 to make sure the scubbing of the name was successful

In [0]:
from pyspark.sql.functions import lit

upsertsDF = (
  spark.read
  .option("versionAsOf", 4)
  .format("delta")
  .load(health_tracker + "processed")
  .where("p_device_id = 4")
  .select("dte", "time", "device_type", "heartrate", lit(None).alias("name"), "p_device_id")
)

In [0]:
processedDeltaTable = DeltaTable.forPath(spark, health_tracker + "processed")

insert = {
  "dte" : "upserts.dte",
  "time" : "upserts.time",
  "device_type" : "upserts.device_type",
  "heartrate" : "upserts.heartrate",
  "name" : "upserts.name",
  "p_device_id" : "upserts.p_device_id"
}

(processedDeltaTable.alias("health_tracker")
 .merge(upsertsDF.alias("upserts"), update_match)
 .whenMatchedUpdate(set=update)
 .whenNotMatchedInsert(values=insert)
 .execute())

In [0]:
print("record count:", health_tracker_processed.count() )

record count: 10920

In [0]:
display(health_tracker_processed.where("p_device_id = 4"))

dte time heartrate name p_device_id 2020-01-01 2020-01-01T00:00:00.000+0000 60.7236962271 null 4 2020-01-01 2020-01-01T01:00:00.000+0000 59.7518357438 null 4 2020-01-01 2020-01-01T02:00:00.000+0000 59.7552762926 null 4 2020-01-01 2020-01-01T03:00:00.000+0000 61.8018342845 null 4 2020-01-01 2020-01-01T04:00:00.000+0000 60.3112488045 null 4 2020-01-01 2020-01-01T05:00:00.000+0000 60.0099058887 null 4 2020-01-01 2020-01-01T06:00:00.000+0000 59.8323375338 null 4 2020-01-01 2020-01-01T07:00:00.000+0000 59.9795666159 null 4 2020-01-01 2020-01-01T08:00:00.000+0000 100.6013295271 null 4 2020-01-01 2020-01-01T09:00:00.000+0000 100.1857471896 null 4 2020-01-01 2020-01-01T10:00:00.000+0000 102.1647369575 null 4 2020-01-01 2020-01-01T11:00:00.000+0000 101.2274148773 null 4 2020-01-01 2020-01-01T12:00:00.000+0000 100.6166457251 null 4 2020-01-01 2020-01-01T13:00:00.000+0000 100.881575671 null 4 2020-01-01 2020-01-01T14:00:00.000+0000 100.6551240729 null 4 2020-01-01 2020-01-01T15:00:00.000+0000 99.8234864486 null 4 2020-01-01 2020-01-01T16:00:00.000+0000 102.1769917951 null 4 2020-01-01 2020-01-01T17:00:00.000+0000 101.1937386946 null 4 2020-01-01 2020-01-01T18:00:00.000+0000 102.3949896757 null 4 2020-01-01 2020-01-01T19:00:00.000+0000 101.21254699240001 null 4 2020-01-01 2020-01-01T20:00:00.000+0000 100.0301043091 null 4 2020-01-01 2020-01-01T21:00:00.000+0000 100.2071551457 null 4 2020-01-01 2020-01-01T22:00:00.000+0000 61.5860196606 null 4 2020-01-01 2020-01-01T23:00:00.000+0000 60.4378692462 null 4 2020-01-02 2020-01-02T00:00:00.000+0000 49.0782528862 null 4 2020-01-02 2020-01-02T01:00:00.000+0000 49.0829638601 null 4 2020-01-02 2020-01-02T02:00:00.000+0000 49.6571661889 null 4 2020-01-02 2020-01-02T03:00:00.000+0000 48.3377914374 null 4 2020-01-02 2020-01-02T04:00:00.000+0000 48.7669146963 null 4 2020-01-02 2020-01-02T05:00:00.000+0000 49.3500741525 null 4 2020-01-02 2020-01-02T06:00:00.000+0000 81.9138032854 null 4 2020-01-02 2020-01-02T07:00:00.000+0000 82.1855194824 null 4 2020-01-02 2020-01-02T08:00:00.000+0000 82.8115361875 null 4 2020-01-02 2020-01-02T09:00:00.000+0000 82.960108924 null 4 2020-01-02 2020-01-02T10:00:00.000+0000 81.805329869 null 4 2020-01-02 2020-01-02T11:00:00.000+0000 80.811161449 null 4 2020-01-02 2020-01-02T12:00:00.000+0000 82.4930348654 null 4 2020-01-02 2020-01-02T13:00:00.000+0000 82.3732406975 null 4 2020-01-02 2020-01-02T14:00:00.000+0000 83.1715551333 null 4 2020-01-02 2020-01-02T15:00:00.000+0000 82.5100242003 null 4 2020-01-02 2020-01-02T16:00:00.000+0000 83.2450831014 null 4 2020-01-02 2020-01-02T17:00:00.000+0000 82.4441340441 null 4 2020-01-02 2020-01-02T18:00:00.000+0000 82.3986329961 null 4 2020-01-02 2020-01-02T19:00:00.000+0000 83.584511034 null 4 2020-01-02 2020-01-02T20:00:00.000+0000 82.7006382938 null 4 2020-01-02 2020-01-02T21:00:00.000+0000 82.632335113 null 4 2020-01-02 2020-01-02T22:00:00.000+0000 83.3651211951 null 4 2020-01-02 2020-01-02T23:00:00.000+0000 48.4260496699 null 4 2020-01-03 2020-01-03T00:00:00.000+0000 60.0690742532 null 4 2020-01-03 2020-01-03T01:00:00.000+0000 59.3915537411 null 4 2020-01-03 2020-01-03T02:00:00.000+0000 58.5686538057 null 4 2020-01-03 2020-01-03T03:00:00.000+0000 59.5155885143 null 4 2020-01-03 2020-01-03T04:00:00.000+0000 59.0719192111 null 4 2020-01-03 2020-01-03T05:00:00.000+0000 58.2473157604 null 4 2020-01-03 2020-01-03T06:00:00.000+0000 98.3938926524 null 4 2020-01-03 2020-01-03T07:00:00.000+0000 98.4183170538 null 4 2020-01-03 2020-01-03T08:00:00.000+0000 98.0384225058 null 4 2020-01-03 2020-01-03T09:00:00.000+0000 99.2706665791 null 4 2020-01-03 2020-01-03T10:00:00.000+0000 97.0575245907 null 4 2020-01-03 2020-01-03T11:00:00.000+0000 97.615330135 null 4 2020-01-03 2020-01-03T12:00:00.000+0000 96.6819814462 null 4 2020-01-03 2020-01-03T13:00:00.000+0000 98.4350090596 null 4 2020-01-03 2020-01-03T14:00:00.000+0000 98.2740693327 null 4 2020-01-03 2020-01-03T15:00:00.000+0000 99.6325442039 null 4 2020-01-03 2020-01-03T16:00:00.000+0000 97.6099398

#### But the name is not *really* gone ...

In [0]:
(spark.read
 .option("versionAsOf", 4)
 .format("delta")
 .load(health_tracker + "processed")
 .where("p_device_id = 4")
 .select("dte", "time", "device_type", "heartrate", "name", "p_device_id")
).show(truncate=False)

+----------+-------------------+-----------+------------------+---------+-----------+
dte |time |device_type|heartrate |name |p_device_id|
+----------+-------------------+-----------+------------------+---------+-----------+
2020-01-01|2020-01-01 00:00:00|null |60.7236962271 |James Hou|4 |
2020-01-01|2020-01-01 01:00:00|null |59.7518357438 |James Hou|4 |
2020-01-01|2020-01-01 02:00:00|null |59.7552762926 |James Hou|4 |
2020-01-01|2020-01-01 03:00:00|null |61.8018342845 |James Hou|4 |
2020-01-01|2020-01-01 04:00:00|null |60.3112488045 |James Hou|4 |
2020-01-01|2020-01-01 05:00:00|null |60.0099058887 |James Hou|4 |
2020-01-01|2020-01-01 06:00:00|null |59.8323375338 |James Hou|4 |
2020-01-01|2020-01-01 07:00:00|null |59.9795666159 |James Hou|4 |
2020-01-01|2020-01-01 08:00:00|null |100.6013295271 |James Hou|4 |
2020-01-01|2020-01-01 09:00:00|null |100.1857471896 |James Hou|4 |
2020-01-01|2020-01-01 10:00:00|null |102.1647369575 |James Hou|4 |
2020-01-01|2020-01-01 11:00:00|null |101.2274148773 |James Hou|4 |
2020-01-01|2020-01-01 12:00:00|null |100.6166457251 |James Hou|4 |
2020-01-01|2020-01-01 13:00:00|null |100.881575671 |James Hou|4 |
2020-01-01|2020-01-01 14:00:00|null |100.6551240729 |James Hou|4 |
2020-01-01|2020-01-01 15:00:00|null |99.8234864486 |James Hou|4 |
2020-01-01|2020-01-01 16:00:00|null |102.1769917951 |James Hou|4 |
2020-01-01|2020-01-01 17:00:00|null |101.1937386946 |James Hou|4 |
2020-01-01|2020-01-01 18:00:00|null |102.3949896757 |James Hou|4 |
2020-01-01|2020-01-01 19:00:00|null |101.21254699240001|James Hou|4 |
+----------+-------------------+-----------+------------------+---------+-----------+
only showing top 20 rows

# Permanently Deleting Data with `VACUUM` 
This command removes files no longer representing the latest state of the table based on the transaction logs and which are older than a threshold (the default is 7 days).

In [0]:
processedDeltaTable.vacuum(0)

--------------------------------------------------------------------------- 
 IllegalArgumentException Traceback (most recent call last)
 <command-1569613669975873> in <module> 
 ----> 1 processedDeltaTable . vacuum ( 0 ) 

 /databricks/spark/python/delta/tables.py in vacuum (self, retentionHours) 
 225 return DataFrame ( jdt . vacuum ( ) , self . _spark . _wrapped ) 
 226 else : 
 --> 227 return DataFrame ( jdt . vacuum ( float ( retentionHours ) ) , self . _spark . _wrapped ) 
 228 
 229 @ since ( 0.4 ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 IllegalArgumentException : requirement failed: Are you sure you would like to vacuum files with such a low retention period? If you have
writers that are currently writing to this table, there is a risk that you may corrupt the
state of your Delta table.

If you are certain that there are no operations being performed on this table, such as
insert/upsert/delete/optimize, then you may turn off this check by setting:
spark.databricks.delta.retentionDurationCheck.enabled = false

If you are not sure, please use a value not less than "168 hours".

### Safeguards against deleting and how to override the safeguards
Initially we will pass the parameter 0, meaning going back 0 days, but that will generate an error message - this is set as a safeguard to prevent removing all of the history.

Below we disable this safeguard and again execute the `vacuum` command.

In [0]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", False)

In [0]:
processedDeltaTable.vacuum(0)

Out[61]: DataFrame[]

In [0]:
(spark.read
 .option("versionAsOf", 4)
 .format("delta")
 .load(health_tracker + "processed")
 .where("p_device_id = 4")
 .select("dte", "time", "device_type", "heartrate", "name", "p_device_id")
).show(truncate=False)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1569613669975877> in <module> 
 5 . where ( "p_device_id = 4" ) 
 6 . select ( "dte" , "time" , "device_type" , "heartrate" , "name" , "p_device_id" ) 
 ----> 7 ).show(truncate=False)
 
 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 441 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 442 else : 
 --> 443 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 
 444 
 445 def __repr__ ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 125 def deco ( * a , ** kw ) : 
 126 try : 
 --> 127 return f ( * a , ** kw ) 
 128 except py4j . protocol . Py4JJavaError as e : 
 129 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o1887.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 261.0 failed 1 times, most recent failure: Lost task 0.0 in stage 261.0 (TID 834, ip-10-172-235-126.us-west-2.compute.internal, executor driver): com.databricks.sql.io.FileReadException: Error while reading file dbfs:/dbacademy/Ramya/DLRS/healthtracker/processed/p_device_id=4/part-00000-66c3054b-e7cf-4f3a-93a0-a2e3939165f3.c000.snappy.parquet. A file referenced in the transaction log cannot be found. This occurs when data has been manually deleted from the file system rather than using the table `DELETE` statement. For more information, see https://docs.databricks.com/delta/delta-intro.html#frequently-asked-questions
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logFileNameAndThrow(FileScanRDD.scala:343)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:305)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.$anonfun$prepareNextFile$1(FileScanRDD.scala:475)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.$anonfun$run$1(SparkThreadLocalForwardingThreadPoolExecutor.scala:104)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingHelper.runWithCaptured(SparkThreadLocalForwardingThreadPoolExecutor.scala:68)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingHelper.runWithCaptured$(SparkThreadLocalForwardingThreadPoolExecutor.scala:54)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.runWithCaptured(SparkThreadLocalForwardingThreadPoolExecutor.scala:101)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.run(SparkThreadLocalForwardingThreadPoolExecutor.scala:104)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:

### Deleted data is now gone, but the remaining data and provenance are still there

In [0]:
(spark.read
 .format("delta")
 .load(health_tracker + "processed")
).show(truncate=False)

+----------+-------------------+-------------+--------------+-----------+-----------+
dte |time |heartrate |name |p_device_id|device_type|
+----------+-------------------+-------------+--------------+-----------+-----------+
2020-01-01|2020-01-01 00:00:00|52.8139067501|Deborah Powell|0 |null |
2020-01-01|2020-01-01 01:00:00|53.9078900098|Deborah Powell|0 |null |
2020-01-01|2020-01-01 02:00:00|52.7129593616|Deborah Powell|0 |null |
2020-01-01|2020-01-01 03:00:00|52.2880422685|Deborah Powell|0 |null |
2020-01-01|2020-01-01 04:00:00|52.5156095386|Deborah Powell|0 |null |
2020-01-01|2020-01-01 05:00:00|53.6280743846|Deborah Powell|0 |null |
2020-01-01|2020-01-01 06:00:00|52.1760037066|Deborah Powell|0 |null |
2020-01-01|2020-01-01 07:00:00|90.0456721836|Deborah Powell|0 |null |
2020-01-01|2020-01-01 08:00:00|89.4695644522|Deborah Powell|0 |null |
2020-01-01|2020-01-01 09:00:00|88.1490304138|Deborah Powell|0 |null |
2020-01-01|2020-01-01 10:00:00|86.3092976213|Deborah Powell|0 |null |
2020-01-01|2020-01-01 11:00:00|86.6672980008|Deborah Powell|0 |null |
2020-01-01|2020-01-01 12:00:00|89.2044154758|Deborah Powell|0 |null |
2020-01-01|2020-01-01 13:00:00|88.8691678316|Deborah Powell|0 |null |
2020-01-01|2020-01-01 14:00:00|87.7252170634|Deborah Powell|0 |null |
2020-01-01|2020-01-01 15:00:00|87.8357187561|Deborah Powell|0 |null |
2020-01-01|2020-01-01 16:00:00|87.1769562648|Deborah Powell|0 |null |
2020-01-01|2020-01-01 17:00:00|88.3403107931|Deborah Powell|0 |null |
2020-01-01|2020-01-01 18:00:00|88.9083484735|Deborah Powell|0 |null |
2020-01-01|2020-01-01 19:00:00|87.921434203 |Deborah Powell|0 |null |
+----------+-------------------+-------------+--------------+-----------+-----------+
only showing top 20 rows

In [0]:
display(processedDeltaTable.history())

version timestamp userId userName operation operationParameters job notebook clusterId readVersion isolationLevel isBlindAppend operationMetrics userMetadata 6 2020-11-14T02:56:29.000+0000 6469525191966356 ramyasree.mallubhatla@sjsu.edu MERGE Map(predicate -> ((health_tracker.`time` = upserts.`time`) AND (health_tracker.`p_device_id` = upserts.`p_device_id`)), matchedPredicates -> [{"actionType":"update"}], notMatchedPredicates -> [{"actionType":"insert"}]) null List(1569613669975775) 1114-024003-chain719 5 WriteSerializable false Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 2, numTargetRowsInserted -> 2184, numTargetRowsUpdated -> 0, numOutputRows -> 2184, numSourceRows -> 2184, numTargetFilesRemoved -> 0) null 5 2020-11-14T02:56:13.000+0000 6469525191966356 ramyasree.mallubhatla@sjsu.edu DELETE Map(predicate -> ["(`p_device_id` = 4)"]) null List(1569613669975775) 1114-024003-chain719 4 WriteSerializable false Map(numRemovedFiles -> 2) null 4 2020-11-14T02:56:04.000+0000 6469525191966356 ramyasree.mallubhatla@sjsu.edu WRITE Map(mode -> Append, partitionBy -> []) null List(1569613669975775) 1114-024003-chain719 3 WriteSerializable true Map(numFiles -> 5, numOutputBytes -> 58669, numOutputRows -> 3720) null 3 2020-11-14T02:48:20.000+0000 6469525191966356 ramyasree.mallubhatla@sjsu.edu MERGE Map(predicate -> ((health_tracker.`time` = upserts.`time`) AND (health_tracker.`p_device_id` = upserts.`p_device_id`)), matchedPredicates -> [{"actionType":"update"}], notMatchedPredicates -> [{"actionType":"insert"}]) null List(1569613669975775) 1114-024003-chain719 2 WriteSerializable false Map(numTargetRowsCopied -> 1439, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, numTargetRowsInserted -> 0, numTargetRowsUpdated -> 1, numOutputRows -> 1440, numSourceRows -> 1, numTargetFilesRemoved -> 1) null 2 2020-11-14T02:47:58.000+0000 6469525191966356 ramyasree.mallubhatla@sjsu.edu MERGE Map(predicate -> ((health_tracker.`time` = upserts.`time`) AND (health_tracker.`p_device_id` = upserts.`p_device_id`)), matchedPredicates -> [{"actionType":"update"}], notMatchedPredicates -> [{"actionType":"insert"}]) null List(1569613669975775) 1114-024003-chain719 1 WriteSerializable false Map(numTargetRowsCopied -> 7068, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 5, numTargetRowsInserted -> 72, numTargetRowsUpdated -> 60, numOutputRows -> 7200, numSourceRows -> 132, numTargetFilesRemoved -> 10) null 1 2020-11-14T02:47:07.000+0000 6469525191966356 ramyasree.mallubhatla@sjsu.edu WRITE Map(mode -> Append, partitionBy -> []) null List(1569613669975775) 1114-024003-chain719 0 WriteSerializable true Map(numFiles -> 5, numOutputBytes -> 52877, numOutputRows -> 3408) null 0 2020-11-14T02:46:16.000+0000 6469525191966356 ramyasree.mallubhatla@sjsu.edu CONVERT Map(numFiles -> 5, partitionedBy -> ["p_device_id"], collectStats -> true) null List(1569613669975775) 1114-024003-chain719 -1 Serializable false Map(numConvertedFiles -> 5) null